In [6]:
import sqlite3
from PyPDF2 import PdfFileReader, PdfFileWriter
import os


def pdf_to_text(pdf_file_path, text_file_name):
    #"turn pdf to text file"
    if '.pdf' not in pdf_file_path:
        pdf_file_path = pdf_file_path + '.pdf'
    
    pdf = PdfFileReader(pdf_file_path)
    
    if '.txt' not in text_file_name:
        text_file_name = text_file_name + '.txt'
    
    with open(text_file_name, 'w') as f:
        for page_num in range(pdf.numPages):
            if page_num == 0 or page_num == 1:
                continue
            
            pageObj = pdf.getPage(page_num)

            try: 
                txt = pageObj.extractText()
            except:
                pass
            else:
                f.write('Page {0}\n'.format(page_num+1))
                f.write(''.center(100, '-'))
                f.write(txt)
        f.close()
        
    return "PDF converted to text file at {}".format(text_file_name)


def extract_cusip(txt_file_path):
    #"find all cusip in text file"
    if '.txt' not in txt_file_path:
        txt_file_path = txt_file_path + '.txt'
    
    with open(txt_file_path) as f:
        text_body = f.readlines()
        
    lst = []
    for i in range(len(text_body)-2):
        if len(text_body[i][:-1]) == 6:
            cusip = text_body[i][:-1] + text_body[i+1][:-1] + text_body[i+2][:-1]

            if cusip.isalnum() and len(cusip) == 9:
                lst.append(cusip)
                         
    return set(lst)


def cusip_to_database(cusip_list, db_file_path):
    #NEED TO ADD PORTION WHERE IT IGNORES CURRENT ID'S (EXCEPT CLAUSE MAY NOT BE SUFFiCIENT)
    #Add cusip to database and classify asset classe
    if '.db' not in db_file_path:
        db_file_path = db_file_path + ".db"
    
    con = sqlite3.connect(db_file_path)
    cursor = con.cursor()
    
    for cusip in cusip_list:
        try:
            if cusip[6:8] == '90':
                cursor.execute("INSERT INTO info VALUES('%s', '%s', '%s', '%s')" % (cusip, 'na', 'na', 'CALL'))
                con.commit()
                continue
                
            if cusip[6:8] == '95':
                cursor.execute("INSERT INTO info VALUES('%s', '%s', '%s', '%s')" % (cusip, 'na', 'na', 'PUT'))
                con.commit()
                continue
                
            cursor.execute("INSERT INTO info VALUES('%s', '%s', '%s', '%s')" % (cusip, 'na', 'na', 'na'))
            con.commit()
            
            
            
        except sqlite3.IntegrityError:
            continue
    
    con.close()
    return "CUSIP list added to database"



def cusip_extraction_wrapper(pdf_file_path, text_file_path, db_file_path):
    pdf_to_text(pdf_file_path, text_file_path)
    x = extract_cusip(text_file_path)
    cusip_to_database(x, db_file_path)
    
    
    os.remove(text_file_path)

In [7]:
pdf_path = r'C:\Users\sbucarion1\london python\Official Python Business\Trading Strategies Spring 2021 Current\13F Scraper\sec_pdfs\13flist2022q1.pdf'
txt_path = 'cusip_list.txt'
db_path = 'cusip.db'

#Convert SEC pdf to a text file
# pdf_to_text(pdf_path, txt_path)

# #Find all CUSIPs within the text file
# results = extract_cusip(txt_path)

# #Add all cusips to the database
# cusip_to_database(results, db_path)

In [8]:
cusip_extraction_wrapper(pdf_path,txt_path,db_path)

In [ ]:
#22079 rows
#23045 rows after q1 pdf updates